In [8]:
import sqlite3

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor

from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.linear_model import LinearRegression



In [9]:
#Format a dataframe after impoting from SQL
def format_df(df_in):
    df=df_in.copy(deep=True)
    df["datetime"] = pd.to_datetime(df["datetime"])
    df=df.set_index('datetime')
    df["date"] = pd.to_datetime(df["date"])
    return df

In [10]:
con = sqlite3.connect(r"C:\Users\dakot\Desktop\DataScience\projects\weather_prediction\NDBC_model_building_database.db")
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('NDBC_historical_raw_data',), ('NDBC_historical_raw_data_St#{STATIONNUMBER}',), ('NDBC_historical_raw_data_St{STATIONNUMBER}',), ('NDBC_historical_raw_data_St46054',), ('NDBC_historical_cleaned_data',), ('NDBC_historical_data_for_training',)]


In [11]:
#Query cleaned data from DB
conn = sqlite3.connect(r"C:\Users\dakot\Desktop\DataScience\projects\weather_prediction\NDBC_model_building_database.db")
df_training = pd.read_sql_query("SELECT * FROM NDBC_historical_data_for_training", conn, index_col=None)

In [12]:
df_training=format_df(df_training)

In [13]:
# class BoostedHybridModel:
#     def __init__(self, model_1, model_2):
#         self.model_1=model_1
#         self.model_2=model_2
#         self.y_column=None

#     def fit(self, X_model_2, y):

#         fourier_pairs=CalendarFourier(freq="A", order=3)
#         X=fourier_pairs.in_sample(X_model_2.index)
#         X['constant']=1
#         X_model_1=X
#         self.model_1.fit(X_model_1, y)
#         y_fit=pd.Series(self.model_1.predict(X_model_1), index=X_model_1.index, name=y.name)
#         y_residuals=y-y_fit
#         self.model_2.fit(X_model_2, y_residuals)
#         self.y_columns=y.name
#         self.y_fit=y_fit
#         self.y_residuals=y_residuals

#     def predict(self, X_model_2):

#         fourier_pairs=CalendarFourier(freq="A", order=3)
#         X=fourier_pairs.in_sample(X_model_2.index)
#         X['constant']=1
#         X_model_1=X
#         y_predict=pd.Series(self.model_1.predict(X_model_1), index=X_model_1.index, name=self.y_column)
#         y_predict+=self.model_2.predict(X_model_2)
#         return y_predict

In [14]:
from BoostedHybridModel import BoostedHybridModel

In [15]:
df_training.columns

Index(['WDIR', 'WSPD', 'GST', 'PRES', 'ATMP', 'WTMP', 'date', 'const', 'trend',
       'sin(1,freq=A-DEC)', 'cos(1,freq=A-DEC)', 'sin(2,freq=A-DEC)',
       'cos(2,freq=A-DEC)', 'sin(3,freq=A-DEC)', 'cos(3,freq=A-DEC)',
       'ATMP_lag_1', 'ATMP_lag_2', 'ATMP_lag_3', 'ATMP_lag_4', 'ATMP_lag_5',
       'ATMP_lag_6', 'ATMP_lag_7', 'ATMP_lag_8', 'WTMP_lag_1', 'WTMP_lag_2',
       'WTMP_lag_3', 'WTMP_lag_4', 'WTMP_lag_5', 'WTMP_lag_6', 'WTMP_lag_7',
       'WTMP_lag_8', 'WSPD_lag_1', 'WSPD_lag_2'],
      dtype='object')

In [16]:

# One day forecast

X_model_2=df_training[['ATMP_lag_1', 'ATMP_lag_2', 'ATMP_lag_3',
       'ATMP_lag_4', 'ATMP_lag_5', 'ATMP_lag_6', 'ATMP_lag_7', 'ATMP_lag_8',
       'WTMP_lag_1', 'WTMP_lag_2', 'WTMP_lag_3', 'WTMP_lag_4', 'WTMP_lag_5',
       'WTMP_lag_6', 'WTMP_lag_7', 'WTMP_lag_8', 'WSPD_lag_1', 'WSPD_lag_2']]

y=df_training['ATMP']

In [17]:
X_model_2_train, X_model_2_test, y_train, y_test = train_test_split( X_model_2, y, test_size=.2, shuffle=False)

In [18]:
model_1=LinearRegression()
model_2=XGBRegressor()

model=BoostedHybridModel(model_1, model_2)

model.fit(X_model_2_train, y_train)

In [19]:
y_pred = model.predict(X_model_2_test)
#y_pred = y_pred.clip(0.0)

In [20]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error( y_test, y_pred)

0.9662063072859188

In [21]:
import pickle

modelFile = open('TemperatureModel_OneDay.p', 'wb')
    
# source, destination
pickle.dump(model, modelFile)                     
modelFile.close()

In [22]:
modelFile = open('TemperatureModel_OneDay.p', 'rb')     
pmodel = pickle.load(modelFile)
y_predp=pmodel.predict(X_model_2_test)
modelFile.close()
mean_absolute_error( y_test, y_predp)

0.9662063072859188

In [23]:
#Two day forecast
X_model_2=df_training[[ 'ATMP_lag_2', 'ATMP_lag_3',
       'ATMP_lag_4', 'ATMP_lag_5', 'ATMP_lag_6', 'ATMP_lag_7', 'ATMP_lag_8',
        'WTMP_lag_2', 'WTMP_lag_3', 'WTMP_lag_4', 'WTMP_lag_5',
       'WTMP_lag_6', 'WTMP_lag_7', 'WTMP_lag_8', 'WSPD_lag_2']]


y=df_training['ATMP']

In [24]:
X_model_2_train, X_model_2_test, y_train, y_test = train_test_split( X_model_2, y, test_size=.2, shuffle=False)

In [25]:
model_1=LinearRegression()
model_2=XGBRegressor()

model=BoostedHybridModel(model_1, model_2)

model.fit(X_model_2_train, y_train)

In [26]:
y_pred = model.predict(X_model_2_test)
#y_pred = y_pred.clip(0.0)

In [27]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error( y_test, y_pred)

1.2087390774034952

In [28]:
import pickle

modelFile = open('TemperatureModel_TwoDay.p', 'wb')
    
# source, destination
pickle.dump(model, modelFile)                     
modelFile.close()

In [29]:
modelFile = open('TemperatureModel_TwoDay.p', 'rb')     
pmodel = pickle.load(modelFile)
y_predp=pmodel.predict(X_model_2_test)
modelFile.close()
mean_absolute_error( y_test, y_predp)

1.2087390774034952